# Reinforcement Learning Solution to the Towers of Hanoi Puzzle

In [2]:
import numpy as np
import copy
from IPython.display import display, clear_output
import random
from random import randint
import matplotlib.pyplot as plt

Valid moves is a generic code for as much no of discs as you like.

In [5]:
def printState(state):
    for i in state:
        for j in i:
            print(j)
    print('------------')
    
def validMoves(state):
    move = [] 
    count1 = 1
    for i in state:
        if i == []:
            count1 = count1 + 1
            continue
        else:
            first = i[-1]
        count = 1
        for locations in state:
            #print(count)
            if locations == i:
                count = count +1
                continue
            else:
                if locations == []:
                    move.append([count1,count])
                else:
                    if locations[-1]<first:
                        count = count + 1
                        continue
                    else:
                        move.append([count1,count])
            count = count +1
        count1 = count1 +1
    return(move)

def makeMove(state, move):
    tempstate = copy.deepcopy(state)
    #print(state)
    #print(move)
    #print(move[0])
    element = state[move[0]-1][0]
    del tempstate[move[0]-1][0]
    #print(tempstate[move[0]][0])
    tempstate[move[1]-1].insert(0,element)
    return(tempstate)
                
def stateMoveTuple(state, move):
    tempmove = copy.deepcopy(move)
    temp = []
    for st in state:
        
        if st != []:
            temp.append(tuple(st))
            #print(temp)
        else:
            temp.append(tuple())
    #print(temp)
    return(tuple([tuple(temp)]) + tuple([tuple(tempmove)]))


def epsilonGreedy(epsilon, Q, state):
    validMove = validMoves(state)
    if np.random.uniform() < epsilon:       
        return validMove[randint(0, len(validMove)-1)]
    else:
        # Greedy Move
        Qs = []
        for m in validMove:
            Qs.append(Q.get(stateMoveTuple(state,m), -1))
        #Qs = np.array([Q.get(stateMoveTuple(state, m), 0) for m in validMove]) 
        return validMove[np.argmax(np.asarray(Qs))]

    
def winner(state):
    Win = [[], [], [1,2,3]]
    if state == Win:
        return True
    else:
        return False

In [36]:
def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMoves, makeMove):
    global Q    
    maxGames = nRepetitions
    rho = learningRate
    epsilonDecayRate = epsilonDecayFactor
    epsilon = 1
    Q = {}   
    Stepstogoal = []
    for nGames in range(maxGames):
        epsilon *= epsilonDecayRate
        step = 0
        state = [[1,2,3], [], []] #  start state
        done = False
        showMoves = False
        while not done:        
            step += 1
            move = epsilonGreedy(epsilon, Q, state)
            stateNew = copy.deepcopy(state)
            stateNew = makeMove(state,move) 
            if winner(stateNew):
                Q[stateMoveTuple(state, move)] = 0
                done = True
            if stateMoveTuple(state, move) not in Q:
                Q[stateMoveTuple(state, move)] = -1  # initial Q value for new state,move
            if step > 1:
                Q[stateMoveTuple(stateOld, moveOld)] += rho * (-1+Q[stateMoveTuple(state, move)] - Q[stateMoveTuple(stateOld, moveOld)])
            stateOld, moveOld = state, move 
            state = stateNew
        Stepstogoal.append(step)
    return Q,Stepstogoal
    

In [51]:
def testQ(Q, maxSteps, validMovesF, makeMoveF):
    done = False
    state = [[1,2,3], [], []] #  start state
    statereturn = []
    while not done:
        
        validmoves = validMoves(state)
        for m in validmoves:
            Qs = np.array([Q.get(stateMoveTuple(state, m), 0) for m in validmoves]) 
        move = validmoves[np.argmax(Qs)]    
        newstate = makeMove(state, move)
        statereturn.append(newstate)
        if winner(newstate):
            done = True
        state = newstate
    return statereturn

In [44]:
Q, stepsToGoal = trainQ(50, 0.5, 0.7, validMoves, makeMove)
print(len(Q))

81


In [52]:
path = testQ(Q, 20, validMoves, makeMove)

In [53]:
path

[[[2, 3], [], [1]],
 [[3], [2], [1]],
 [[3], [1, 2], []],
 [[], [1, 2], [3]],
 [[1], [2], [3]],
 [[1], [], [2, 3]],
 [[], [], [1, 2, 3]]]